In [234]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps

%matplotlib inline

In [235]:
def euclidean_proj_simplex(v, s=1):
    n, = v.shape  # will raise ValueError if v is not 1-D
    # check if we are already on the simplex
    if v.sum() == s and np.alltrue(v >= 0):
        # best projection: itself!
        return v
    # get the array of cumulative sums of a sorted (decreasing) copy of v
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u)
    # get the number of > 0 components of the optimal solution
    rho = np.nonzero(u * np.arange(1, n+1) > (cssv - s))[0][-1]
    # compute the Lagrange multiplier associated to the simplex constraint
    theta = (cssv[rho] - s) / (rho + 1.0)
    # compute the projection by thresholding v using theta
    w = (v - theta).clip(min=0)
    return w


def ternary_search(func, left, right, precision):
    # finds argmax of convex function f
    # on the segment [left; right] with given precision
    if abs(right - left) < precision:
        return left

    left_third = (2*left + right)/3
    right_third = (left + 2*right)/3
    if func(left_third) == func(right_third):
        return ternary_search(func, left_third, right_third, precision) 
    if func(left_third) < func(right_third):
        return ternary_search(func, left_third, right, precision) 
    else:
        return ternary_search(func, left, right_third, precision)


def line_search(f, precision=1e-7, estimate=0):
    # finds argmax of convex function f
    # on the line with given precision
    curr = estimate
    delta = 1
    while f(curr + delta) > f(curr):
        curr = curr + delta
        delta *= 2
    curr_left = estimate
    delta_left = 1
    while f(curr_left - delta_left) > f(curr_left):
        curr_left = curr_left - delta_left
        delta_left *= 2
    return ternary_search(f, curr_left - delta_left, curr + delta, precision)


In [256]:
# Gradient descent method

def proj_grad_descent(x0, precision, max_iters):
    cur_x = x0
    previous_step_size = 1
    iters = 0
    while previous_step_size > precision and iters < max_iters:
        prev_x = cur_x
        grad = df(prev_x)
        
        def func(alpha):
            return f(euclidean_proj_simplex(prev_x + alpha * grad))
        alpha = line_search(func)

        cur_x = euclidean_proj_simplex(prev_x + alpha * grad)
        previous_step_size = np.linalg.norm(cur_x - prev_x)
        iters += 1
        
    np.set_printoptions(precision=4)
    print("The local maximum of proj grad occurs at", cur_x)
    print("Number of gradient iterations: ", iters)
    return cur_x


# Quasi-Newton method

def proj_quasi_newton_method (x0, precision, max_iters):
    # DFP algorithm
    cur_x = x0
    previous_step_size = 1
    iters = 0
    H = np.eye(np.size(x0))
    alpha = 1
    while previous_step_size > precision and iters < max_iters:
        prev_x = cur_x
        h = -H @ df(prev_x)
        
        def func(alpha):
            return f(euclidean_proj_simplex(prev_x + alpha * h))
        
        alpha = line_search(func)
        
        cur_x = euclidean_proj_simplex(prev_x + alpha * h)
        s = np.atleast_2d(cur_x - prev_x).T
        y = np.atleast_2d(df(cur_x) - df(prev_x)).T
        
        H = H - (H @ y @ y.T @ H) / (y.T @ H @ y) + (s @ s.T) / (y.T @ s)
        previous_step_size = np.linalg.norm(cur_x - prev_x) 
        iters += 1
        
    np.set_printoptions(precision=4)
    print("The local maximum of quasi newton occurs at", cur_x)
    print("Number of quasi-newton iterations: ", iters)
    return cur_x


# Newton method 

def proj_newton_method (x0, precision, max_iters):
    cur_x = x0
    previous_step_size = 1
    iters = 0
    while previous_step_size > precision and iters < max_iters:
        prev_x = cur_x
        h = - np.linalg.inv(ddf(prev_x)) @ df(prev_x)
        def func(alpha):
            return f(euclidean_proj_simplex(prev_x + alpha * h))
        
        alpha = line_search(func)
        cur_x = euclidean_proj_simplex(prev_x + alpha * h)
        previous_step_size = np.linalg.norm(cur_x - prev_x) 
        iters += 1
        
    np.set_printoptions(precision=4)
    print("The local maximum of newton occurs at", cur_x)
    print("Number of newton iterations: ", iters)
    return cur_x


# Interior point method

def interior_point_method(x0, t0, alpha, precision, max_iters):   
    n = x0.size
    A = np.atleast_2d(np.ones(n))
    b = np.ones(1)
    cur_x = x0
    t = t0
    
    def phi(x, t):
        if np.min(x) <= 0:
            return -np.inf
        return f(x) + t * np.sum(np.log(x))
    
    def phi_grad(x, t):
        return df(x) + t/x
    
    def phi_hess(x, t):
        return ddf(x) - t * np.diag(1/(x**2))
    
    def newton_equality_feasible(x0, A, b, t):
        # solves maximization problem for phi(x, t) with constraints Ax = b 
        cur_x = x0
        m = A.shape[0]
        n = A.shape[1]
        while True:
            newton_matrix = np.bmat([[-phi_hess(cur_x, t), A.T], [A, np.zeros((m, m))]])
            rhs = np.atleast_2d(np.hstack([phi_grad(cur_x, t), np.zeros(m)])).T
            w = np.linalg.solve(newton_matrix, rhs)
            h = w[:n]
            if np.abs(h.T @ phi_hess(cur_x, t) @ h) < precision:
                break
            
            h = h.reshape(n)
            
            def func(alpha):
                return phi(cur_x + alpha * h, t)
            
            alpha = line_search(func)
            cur_x = cur_x + alpha * h
        return cur_x
   
    while True:
        cur_x = newton_equality_feasible(cur_x, A, b, t)
        if n * t < precision:
            break
        t *= alpha
    print("The local maximum of interior point method occurs at", cur_x)
    return cur_x
    

In [257]:
# Initialization

n = 5
m = 5

P = np.random.rand(n, m) 
P /= P.sum(axis=1)[:,None]
P = P.T
c = (P * np.log2(P)).sum(axis=0)

def f(x):
    array = (P @ x) * np.log2(P @ x)
    return c @ x - array.sum() 

def df(x):
    k = P.shape[0]
    array = np.array([P[i] * (np.log2(P[i] @ x) + 1/np.log(2)) for i in range(k)])
    return c - array.sum(axis=0)

def ddf(x):
    k = P.shape[0]
    t = P @ x
    array = np.array([np.atleast_2d(P[i]).T @ np.atleast_2d(P[i]) for i in range(k)])
    result = - np.array([array[i] / (np.log(2) * t[i]) for i in range(k)]).sum(axis=0)
    return result

In [258]:
x0 = np.random.rand(5)
x0 /= x0.sum()
precision = 10**-6
iters = 10**3

print('Starting point:', x0)

x1 = proj_grad_descent(x0, precision, iters)
print("max proj grad capacity:", f(x1), '\n')
x2 = proj_quasi_newton_method (x0,  precision, iters)
print("max proj quasi newton capacity:", f(x2), '\n')
x3 = proj_newton_method (x0, precision, iters)
print("max proj newton capacity:", f(x3), '\n')
x4 = interior_point_method(x0, 1, 0.1, precision, iters)
print("max interior point method capacity:", f(x4))


Starting point: [ 0.2923  0.0943  0.2133  0.2076  0.1925]
The local maximum of proj grad occurs at [ 0.0117  0.      0.4703  0.      0.518 ]
Number of gradient iterations:  29
max proj grad capacity: 0.246013234784 

The local maximum of quasi newton occurs at [  2.3636e-01   1.3711e-09   2.1548e-01   1.4127e-01   4.0689e-01]
Number of quasi-newton iterations:  5
max proj quasi newton capacity: 0.223841316672 

The local maximum of newton occurs at [ 0.4846  0.      0.1421  0.      0.3733]
Number of newton iterations:  5
max proj newton capacity: 0.215658548458 

The local maximum of interior point method occurs at [  1.1978e-02   5.5742e-07   4.7010e-01   8.1303e-07   5.1792e-01]
max interior point method capacity: 0.24601302442
